In [ ]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np
import math

from utils.data_dirs import data_dirs
from utils.get_camera_heading import get_camera_heading
from utils.get_camera_position import get_camera_position
from utils.get_camera_rotation import get_camera_rotation
from utils.get_heading_from_north import get_heading_from_north
from utils.get_heading import get_heading
from utils.index import index
from utils.normalize_angle import normalize_angle
from utils.unique import unique

In [ ]:
EXPERIMENT = False

In [ ]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

In [ ]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

In [ ]:
if EXPERIMENT:
    files = os.listdir(EXPERIMENT_DATA)
    files_set = set(files)
    sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
else:
    sample_data_filter = sample_data_json
sample_data_filter = [s for s in sample_data_filter if s['fileformat'] == 'jpg']

In [ ]:
sample_data_filter[0]

In [ ]:
sample_tokens = unique(sample_data_filter, 'sample_token')
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token']
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

In [ ]:
calibrated_sensor_tokens = unique(sample_data_filter, 'calibrated_sensor_token')
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

In [ ]:
ego_pose_tokens = unique(sample_data_filter, 'ego_pose_token')
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation']
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

In [ ]:
scene_tokens = unique(sample_filter, 'scene_token')
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

In [ ]:
sample_map = index(sample_filter, 'sample_token')
calibrated_sensor_map = index(calibrated_sensor_filter, 'calibrated_sensor_token')
ego_pose_map = index(ego_pose_filter, 'ego_pose_token')
scene_map = index(scene_filter, 'scene_token')

def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    scene = scene_map[sample['scene_token']]
    
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene,
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalize_angle(camera_heading + ego_heading) * 180 / math.pi,
        'camera_translation': get_camera_position(
            calibrated_sensor['camera_translation'],
            ego_pose['ego_translation'],
            ego_pose['ego_rotation'],
        ),
        'camera_rotation': get_camera_rotation(
            calibrated_sensor['camera_rotation'],
            ego_pose['ego_rotation'],
        ),
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    return ret

sample_data_res = [*map(s_map, sample_data_filter)]

len(sample_data_res)

In [ ]:
sample_data_res[0]

In [ ]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

In [ ]:
instance_tokens = unique(sample_annotation_filter, 'instance_token')
instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

In [ ]:
category_tokens = unique(instance_filter, 'category_token')
category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

In [ ]:
instance_map = index(instance_filter, 'instance_token')
category_map = index(category_filter, 'category_token')

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

In [ ]:
sample_data_final = pd.DataFrame(sample_data_res)
sample_data_final[:1]

In [ ]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)
sample_annotation_final[:1]

In [ ]:
len(sample_annotation_final)

In [ ]:
df_sample_data_keyframe = sample_data_final[sample_data_final["is_key_frame"]][
    ["token", "sample_token"]
]
sample_annotation_final = sample_annotation_final.set_index("sample_token").join(
    df_sample_data_keyframe.set_index("sample_token"), on="sample_token", rsuffix="_sample_data"
)
sample_annotation_final[:1]

In [ ]:
len(sample_annotation_final)

In [ ]:
len({*df_sample_data_keyframe["token"].tolist()})

In [ ]:
len({*df_sample_data_keyframe["sample_token"].tolist()})

In [ ]:
sample_data_final = sample_data_final.sort_values(by=['scene_name', 'timestamp'])

In [ ]:
sample_data_final["frame_order"] = 0
scene = ""
i = 1
for index, row in sample_data_final.iterrows():
    if row['scene_name'] != scene:
        scene = row['scene_name']
        sample_data_final.loc[index, "frame_order"] = 1
        i = 2
    else:
        sample_data_final.loc[index, "frame_order"] = i
        i += 1


In [ ]:
print(len(sample_data_final))

In [ ]:
print(len(sample_annotation_final))

In [ ]:
with open(os.path.join(output_dir, f"sample_data{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_data_final, f)
with open(os.path.join(output_dir, f"annotation{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_annotation_final, f)

In [ ]:
sample_data_final.to_csv(os.path.join(output_dir, f"sample_data{suffix}.csv"), index=False)

In [ ]:
sample_annotation_final.to_csv(os.path.join(output_dir, f"annotation{suffix}.csv"), index=False)